# Import libraries

In [1]:
from src.preprocessing.dataLoader_CelebA import get_partitioned_dataloaders, create_subset_loader
from src.ml.resNet50 import SiameseResNet
import torch
from src.ml.hyperparam_study import run_optuna_study

# Import losses

In [2]:
from pytorch_metric_learning.losses import ContrastiveLoss
from pytorch_metric_learning.losses import MarginLoss
from pytorch_metric_learning.losses import MultiSimilarityLoss
from pytorch_metric_learning.losses import HistogramLoss

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.7.0+cu118
11.8
True
NVIDIA GeForce RTX 2070 SUPER


# 1. Load the data

In [4]:
IMAGE_DIR = "data/celeba/img_align_celeba"
LABEL_FILE = "data/celeba/identity_CelebA.txt"
PARTITION_FILE = "data/celeba/list_eval_partition.csv"
IMG_SIZE = 224
BATCH_SIZE = 128
M_PER_SAMPLE = 16
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_loader, val_loader, test_loader = get_partitioned_dataloaders(image_dir= IMAGE_DIR,
                                                               label_file= LABEL_FILE,
                                                                m_per_sample = M_PER_SAMPLE,
                                                               partition_file= PARTITION_FILE,
                                                               batch_size=BATCH_SIZE)

# Create the model

In [6]:
model = SiameseResNet()

# Find best Hyperparameters

In [ ]:
train_loader_study = (create_subset_loader(train_loader,50000))
val_loader_study = (create_subset_loader(train_loader,8000))
study = run_optuna_study(train_loader_study, val_loader_study, n_trials=50, study_name="siamese_constrastive_study", criterion="contrastive")
best_params = study.best_params

[I 2025-05-29 00:19:18,001] Using an existing study with name 'siamese_constrastive_study' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]


Epoch 1/10: 100%|██████████| 391/391 [02:58<00:00,  2.20it/s, batch_loss=1.39179444]

Epoch 1/10 - Validation: 100%|██████████| 63/63 [00:26<00:00,  2.39it/s]


Epoch 0 - Training Loss: 0.83419957, Validation Loss: 0.89052190
Validation loss decreased (inf --> 0.890522). Saving model ...



Epoch 2/10: 100%|██████████| 391/391 [02:54<00:00,  2.24it/s, batch_loss=0.02638659]

Epoch 2/10 - Validation: 100%|██████████| 63/63 [00:26<00:00,  2.38it/s]


Epoch 1 - Training Loss: 0.87847441, Validation Loss: 0.87263530
Validation loss decreased (0.890522 --> 0.872635). Saving model ...



Epoch 3/10:  53%|█████▎    | 206/391 [01:31<01:21,  2.26it/s, batch_loss=1.29367447]

In [ ]:
train_loader_study = (create_subset_loader(train_loader,50000))
val_loader_study = (create_subset_loader(train_loader,8000))
study = run_optuna_study(train_loader_study, val_loader_study, n_trials=50, study_name="siamese_constrastive_study", criterion="ms")
best_params = study.best_params

# Train Model

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3, weight_decay=1e-2)

In [8]:
contrastive_loss = ContrastiveLoss(neg_margin=1.0, pos_margin=0)
histogram_loss = HistogramLoss(n_bins=1000)
multi_similarity_loss = MultiSimilarityLoss()
margin_loss = MarginLoss(margin=0.5)

In [9]:
results = model.train_model(
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=contrastive_loss,
        optimizer=optimizer,
        num_epochs=10,
        device=DEVICE,
        patience=5,
        experiment_name='SiameseResNet',
        tuning_mode=False
    )

Epoch 1/10:   7%|▋         | 86/1271 [00:37<08:39,  2.28it/s, batch_loss=0.98682928]


KeyboardInterrupt: 

In [ ]:
train_loader_study = (create_subset_loader(train_loader,10000))
val_loader_study = (create_subset_loader(train_loader,2000))
study = run_optuna_study(train_loader_study, val_loader_study, n_trials=10, study_name="siamese_constrastive_HP_study")
best_params = study.best_params

# Plot the results

In [ ]:
! mlflow ui --port 5000